In [ ]:
!poetry update -q

In [ ]:
from dotenv import load_dotenv

from configurations.chat_openai import *
from configurations.cypher_chat_openai import *
from configurations.default import *
from configurations.neo4j import *
from configurations.openai_embeddings import *

load_dotenv()
configurations = Configuration.load('configuration.yaml')

default = DefaultConfiguration.grab(configurations, '')
neo4j = Neo4jConfiguration.grab(configurations)
chat_openai = ChatOpenAIConfiguration.grab(configurations)
openai_embeddings = OpenAIEmbeddingsConfiguration.grab(configurations)
cypher_chat_openai = CypherChatOpenAIConfiguration.grab(configurations)

request = "The manager of chatbot's project has been changed, now it's Ivan Petrov"
# request = 'Who is the manager of this chatbot project'
# request = 'Vanya Petrov is his own enemy'
# request = 'Vanya Petrov is now his own enemy'

In [ ]:
from langchain_openai import *
from package.utilities import *
from langchain.schema import HumanMessage, SystemMessage

chat = ChatOpenAI(**chat_openai.dict())
orchestrator_prompt = get_file_contents('orchestrator-prompt.txt')
message = chat.invoke([
    SystemMessage(content=orchestrator_prompt),
    HumanMessage(content=request)
])
candidates = [candidate.lstrip('0123456789.- ').strip() for candidate in message.content.splitlines()]
candidates

In [ ]:
from langchain_openai import *
from package.utilities import *
from langchain.schema import HumanMessage, SystemMessage

is_change = ChatOpenAI(**chat_openai.dict()).invoke([
    SystemMessage(content=f"""
    Determine whether the input request is a request to receive any information or to change and give only one of the 2 values: 'read' or 'change'.
    Accept of a declarative-style request as a change request.
    """),
    HumanMessage(content=request)
]).content.strip() == 'change'
is_change

In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import *
from langchain_openai import OpenAIEmbeddings
from neo4j import GraphDatabase

graph = Neo4jGraph(**neo4j.dict())

all_labels = [l['label'] for l in graph.query(
    """
    MATCH (l)
    UNWIND labels(l) AS label
    RETURN DISTINCT label
    """)]
print('All Neo4j node labels', all_labels)

similarity_labels: list[str] = []
if len(all_labels) > 0:
    retriever = FAISS.from_texts(all_labels, OpenAIEmbeddings(**openai_embeddings.dict())).as_retriever()
    similarity_labels = list(set([retriever.invoke(candidate)[0].page_content for candidate in candidates]))
print('Similarity labels from request', similarity_labels)

name = 'name'
title = 'title'

set_instances: set[str] = set()
graph_database = {'uri': neo4j.url, 'auth': (neo4j.username, neo4j.password)}
with GraphDatabase.driver(**graph_database) as driver:
    for similarity_label in similarity_labels:
        index_name = f"{similarity_label.lower()}_index"
        driver.execute_query(f"DROP INDEX {index_name} IF EXISTS")
        store = Neo4jVector.from_existing_graph(
            OpenAIEmbeddings(**openai_embeddings.dict()),
            **neo4j.dict(),
            index_name=index_name,
            node_label=similarity_label,
            text_node_properties=[name, title],
            embedding_node_property='embedding',
        )
        for candidate in candidates:
            response = store.similarity_search(candidate, k=1)
            for string in process_response(response, name, title):
                set_instances.add(string)
            # print(candidate, similarity_label, '->', list(process_response(response, name, title))[0])
similarity_words = list(set_instances) + similarity_labels
similarity_words

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain

graph = Neo4jGraph(**neo4j.dict())
response = dict()

print(is_change)
if is_change:
    CYPHER_GENERATION_TEMPLATE = f"""Task:Generate Cypher statement to query a graph database.
    Instructions:
    Generate query just to change, not get information.
    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.
    Use only similarity words, excluding relations, from this list: {similarity_words}
    Schema:
    {{schema}}
    Note: Do not include any explanations or apologies in your responses.
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    Do not include any text except the generated Cypher statement.
    
    The question is:
    {{question}}"""

    CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=['schema', 'question'], validate_template=True, template=CYPHER_GENERATION_TEMPLATE)

    chain = GraphCypherQAChain.from_llm(
        ChatOpenAI(**cypher_chat_openai.dict()),
        cypher_prompt=CYPHER_GENERATION_PROMPT,
        graph=graph,
        verbose=True,
        allow_dangerous_requests=True,
    )
    # embeddings4o = ChatOpenAI(model=default.chat_openai_model, openai_api_key=OPENAI_API_KEY, temperature=0)
    # vector_store = Neo4jVector(embeddings=embeddings4o, graph=graph)
    # llm = ChatOpenAI(model='gpt-4o', temperature=0, openai_api_key=OPENAI_API_KEY)
    # chain1 = GraphCypherQAChain.from_llm(vector_store=vector_index, llm=llm, verbose=True)

    response = chain.invoke({"query": request})
    # response = chain.invoke({"query": "Appoint Ivan Petrov as chatbot project manager"})
    # response = chain.invoke({"query": "Remove all relations"})
    # response = chain.invoke({"query": "Set relation: term with name manager can works in term with name department"})
else:
    CYPHER_GENERATION_TEMPLATE = f"""Task:Generate Cypher statement to query a graph database.
    Instructions:
    Generate query just to get information, not changes.
    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.
    Use only similarity words from this list: {similarity_words}
    Schema:
    {{schema}}
    Note: Do not include any explanations or apologies in your responses.
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    Do not include any text except the generated Cypher statement.
    
    The question is:
    {{question}}"""

    CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=['schema', 'question'], validate_template=True, template=CYPHER_GENERATION_TEMPLATE)

    chain = GraphCypherQAChain.from_llm(
        ChatOpenAI(**cypher_chat_openai.dict()),
        cypher_prompt=CYPHER_GENERATION_PROMPT,
        graph=graph,
        verbose=True,
        allow_dangerous_requests=True,
    )
    response = chain.invoke({"query": request})
response

In [ ]:
from uuid import uuid4
from peewee import *

db = SqliteDatabase('database.sqlite')

class BaseModel(Model):
    class Meta:
        database = db
        legacy_table_names = False

class Entity(BaseModel):
    id = UUIDField(primary_key=True, default=uuid4)
    type = CharField()  # entity title from Neo4j
    title = CharField()

class Relation(BaseModel):
    from_ = ForeignKeyField(Entity, backref='relations', db_column='from')
    to = ForeignKeyField(Entity, backref='relations', db_column='to')
    type = CharField()  # relation name from §

    class Meta:
        primary_key = CompositeKey('from_', 'to', 'type')

db.connect(True)
db.create_tables([Entity, Relation])
Entity.create_table()
# a = Entity.select().count()
# print(a)    

In [ ]:
# Entity.create(title='New one', type='3db3c5bc-5431-425c-ae2a-00e4603a4578')
# Entity.create(title='Old two', type='4db3c5bc-5431-425c-ae2a-00e4603a4578')
# Relation.create(from_='3db3c5bc-5431-425c-ae2a-00e4603a4578', to='4db3c5bc-5431-425c-ae2a-00e4603a4578', type='Friend')
# Relation.create(from_='4db3c5bc-5431-425c-ae2a-00e4603a4578', to='4db3c5bc-5431-425c-ae2a-00e4603a4578', type='Friend')

In [ ]:
from package.table_generator import *

attach_table_generator(db)

dataFoo = db.generate_table(Entity, 'foo2', {'new_field': CharField()})
# dataBar = db.generate_table(Relation, 'bar2')